## MBAI Homework

The goal is to return pandas dataframes with the results of the queries. You will use the python connector from Snowflake to connect to the database and load the resolved data into pandas dataframes. 

You will submit your homework as a Jupyter notebook with the code and the results of your queries. 

In [9]:
from connection import connect_to_snowflake
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

# Snowflake Connection

In [10]:
connector = connect_to_snowflake(database="SNOWFLAKE_SAMPLE_DATA", warehouse="COMPUTE_WH", schema="TPCH_SF1")
cursor = connector.cursor()

## I. Serverless OLAP Speed

Let's demonstrate the power of Snowflake & Modern OLAP systems. 

There is a table named `lineitem` in the TPCH_SF1 schema. The table `lineitem` has a column named `l_quantity`, which represents the quantity of items sold in that transaction. The table has **6 million rows**. Let's explore two different approaches to get the average quantity sold.

### Slow Approach

The easiest approach to calculating the average quantity is to download the entire table and then calculate the average using pandas. This is a **slow** and **expensive** approach, as it requires downloading all 6 million rows of data. For larger datasets, this approach is not feasible as you will run out of memory in your local setup.

In [25]:
SLOW_QUERY = "select l_quantity from lineitem;"
df = pd.read_sql(SLOW_QUERY, connector)
avg_quantity = float(df["L_QUANTITY"].mean())

# Format into a nice table.
df = pd.DataFrame({"AVG_QUANTITY": [avg_quantity]})
df

,AVG_QUANTITY
0,25.507967


### Fast Approach

The fast approach is to use the Snowflake SQL engine to calculate the average quantity sold. This approach is much faster and more efficient, as it only requires downloading a small amount of data and the computation is done on the (scalable) Snowflake server. This does incur a cost, but ultimately it will be cheaper to operate than a large local server.

In [24]:
FAST_QUERY = "select AVG(l_quantity) as avg_quantity from lineitem;"
df = pd.read_sql(FAST_QUERY, connector)
df

,AVG_QUANTITY
0,25.507967


## II. Outlier Detection

### Outlier Detection with IQR

The Interquartile Range (IQR) is a measure of statistical dispersion and is used to detect outliers in a dataset. The IQR is calculated as the difference between the 75th percentile (Q3) and the 25th percentile (Q1) of the data. Any data point that falls below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR is considered an outlier.

Below, we will use the IQR method to detect outliers in the `o_totalprice` column of the `orders` table.

In [ ]:
# Define the SQL query to detect outliers using the IQR method
OUTLIER_IQR_QUERY = """
WITH Quantiles AS (
  -- CTE to calculate the 25th percentile (Q1) and 75th percentile (Q3) of the o_totalprice column
  SELECT
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY o_totalprice) AS Q1,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY o_totalprice) AS Q3
  FROM orders
),

IQR_Calculation AS (
  -- CTE to calculate the Interquartile Range (IQR) as the difference between Q3 and Q1
  SELECT
    Q1,
    Q3,
    Q3 - Q1 AS IQR
  FROM Quantiles
)

-- Select orders where the o_totalprice is an outlier
SELECT
  l.o_orderkey,
  o_totalprice
FROM
  orders l
WHERE
  -- An outlier is defined as a value below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR
  o_totalprice < (SELECT Q1 - 1.5 * IQR FROM IQR_Calculation) OR o_totalprice > (SELECT Q3 + 1.5 * IQR FROM IQR_Calculation)
"""

df = pd.read_sql(OUTLIER_IQR_QUERY, connector)
df

,O_ORDERKEY,O_TOTALPRICE
0,8516,430322.22
1,12804,448614.53
2,23684,425765.38
3,29158,443576.50
4,29378,436890.26
...,...,...
1402,2971972,456738.58
1403,2973444,476888.83
1404,2978211,431083.54
1405,2996450,441192.17


### Outlier Detection with Z-Score

The Z-score is a measure of how many standard deviations a data point is from the mean. A Z-score of 0 indicates that the data point is exactly at the mean, while a Z-score of 1 indicates that the data point is one standard deviation above the mean. Any data point with a Z-score greater than 3 or less than -3 is considered an outlier.

In [75]:
OUTLIER_ZSCORE_QUERY = """
WITH Stats AS (
  SELECT
    AVG(o_totalprice) AS mean,
    STDDEV(o_totalprice) AS stddev
  FROM orders
)
SELECT
  o_orderkey,
  o_totalprice
FROM
  orders
WHERE
  ABS(o_totalprice - (SELECT mean FROM Stats)) / (SELECT stddev FROM Stats) > 3
"""
df = pd.read_sql(OUTLIER_ZSCORE_QUERY, connector)
df

,O_ORDERKEY,O_TOTALPRICE
0,8516,430322.22
1,12804,448614.53
2,23684,425765.38
3,29158,443576.50
4,29378,436890.26
...,...,...
1713,3583616,473031.66
1714,3586919,522644.48
1715,3593862,440883.71
1716,3595524,418047.51
